In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try:
    import google.colab
    colab = True
except:
    colab = False

In [3]:
if colab is True:
    # Running in Google Colab
    # Clone the repo
    !git clone https://github.com/sicara/easy-few-shot-learning
    %cd easy-few-shot-learning
    !pip install .
else:
    # Run locally
    # Ensure working directory is the project's root
    # Make sure easyfsl is installed!
    %cd ..

Cloning into 'easy-few-shot-learning'...
remote: Enumerating objects: 1064, done.
remote: Counting objects: 100% (323/323), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 1064 (delta 210), reused 152 (delta 146), pack-reused 741
Receiving objects: 100% (1064/1064), 2.28 MiB | 9.55 MiB/s, done.
Resolving deltas: 100% (617/617), done.
/content/easy-few-shot-learning
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/easy-few-shot-learning
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for easyfsl: filename=easyfsl-1.3.0-py3-none-any.whl size=52364 sha256=8c3a084e54cfe1ddcfb2d8de1e43bcf82507158d75197fa8db319194af326aec
  Stored in directory: /root/.cache/pip/wheels/bb/79/35/1da2d9ffa9c9b6892f0cca4cda77ee397b97a1deda949049e2
Successfully built easyfsl


In [4]:
from pathlib import Path
import random
from statistics import mean

import numpy as np
import torch
from torch import nn
from tqdm import tqdm
import torchvision
import torch.utils.data

In [5]:
random_seed = 0
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [6]:
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True)

100%|██████████| 187490228/187490228 [00:01<00:00, 109264482.57it/s]


Extracting /content/data/gtsrb/GTSRB-Training_fixed.zip to /content/data/gtsrb


In [7]:
!mkdir /content/data/gtsrb/GTSRB/Testing

!mv /content/data/gtsrb/GTSRB/Training/00021 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00037 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00042 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00016 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00006 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00005 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00004 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00035 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00000 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00029 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00010 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00012 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00040 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00009 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00036 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00031 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00041 /content/data/gtsrb/GTSRB/Testing
!mv /content/data/gtsrb/GTSRB/Training/00018 /content/data/gtsrb/GTSRB/Testing

In [8]:
image_size = 28

# Instantiate the datasets
train_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='train',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=False)

train_set_size = int(len(train_set) * 0.8)
val_set_size = len(train_set) - train_set_size
train_set, val_set = torch.utils.data.random_split(train_set, [train_set_size, val_set_size])

train_set.get_labels = lambda: [
    instance[1] for instance in train_set
]

val_set.get_labels = lambda: [
    instance[1] for instance in val_set
]

In [9]:
from torch.utils.data import DataLoader

batch_size = 128
n_workers = 0

# train_set = CUB(split="train", training=True)
train_loader = DataLoader(
    train_set,
    batch_size=batch_size,
    num_workers=n_workers,
    pin_memory=True,
    shuffle=True,
)

In [10]:
from easyfsl.modules import resnet12

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

model = resnet12(
    use_fc=True,
    num_classes=len(set(train_set.get_labels())),
).to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [11]:
from easyfsl.methods import PrototypicalNetworks,Finetune
from easyfsl.samplers import TaskSampler

n_way = 5
n_shot = 5
n_query = 10
n_validation_tasks = 500

# val_set = CUB(split="val", training=False)
val_sampler = TaskSampler(
    val_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

few_shot_classifier = Finetune(model).to(DEVICE)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [12]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter


LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 200
scheduler_milestones = [150, 180]
scheduler_gamma = 0.1
learning_rate = 1e-01
tb_logs_dir = Path(".")

train_optimizer = SGD(
    model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

In [13]:
def training_epoch(model_: nn.Module, data_loader: DataLoader, optimizer: Optimizer):
    all_loss = []
    model_.train()
    with tqdm(data_loader, total=len(data_loader), desc="Training") as tqdm_train:
        for images, labels in tqdm_train:
            optimizer.zero_grad()

            loss = LOSS_FUNCTION(model_(images.to(DEVICE)), labels.to(DEVICE))
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [14]:
from easyfsl.methods.tim import FewShotClassifier
from easyfsl.utils import evaluate


best_state = model.state_dict()
best_validation_accuracy = 0.0
validation_frequency = 10
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    few_shot_classifier.backbone.requires_grad_(True)
    average_loss = training_epoch(model, train_loader, train_optimizer)

    if epoch % validation_frequency == validation_frequency - 1:

        # We use this very convenient method from EasyFSL's ResNet to specify
        # that the model shouldn't use its last fully connected layer during validation.
        model.set_use_fc(False)
        few_shot_classifier.backbone.requires_grad_(False)
        validation_accuracy = evaluate(
            few_shot_classifier, val_loader, device=DEVICE, tqdm_prefix="Validation"
        )
        model.set_use_fc(True)
        few_shot_classifier.backbone.requires_grad_(True)

        if validation_accuracy > best_validation_accuracy:
            best_validation_accuracy = validation_accuracy
            best_state = model.state_dict()
            print("Ding ding ding! We found a new best model!")

        tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)
        # print(validation_accuracy)

    tb_writer.add_scalar("Train/loss", average_loss, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()
    if epoch % 10 == 0 : torch.save(best_state, '/content/drive/MyDrive/pr_final_presentation/weights/Finetune_5_5.pth')

Epoch 0


Training:   0%|          | 0/100 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
Training: 100%|██████████| 100/100 [00:25<00:00,  3.95it/s, loss=1.35]


Epoch 1


Training: 100%|██████████| 100/100 [00:14<00:00,  6.74it/s, loss=0.115]


Epoch 2


...

Training: 100%|██████████| 100/100 [00:15<00:00,  6.47it/s, loss=0.00167]


Epoch 198


Training: 100%|██████████| 100/100 [00:15<00:00,  6.49it/s, loss=0.00155]


Epoch 199


Validation: 100%|██████████| 500/500 [00:32<00:00, 15.28it/s, accuracy=0.999]


In [15]:
model.load_state_dict(torch.load('/content/drive/MyDrive/pr_final_presentation/weights/Finetune_5_5.pth'))

<All keys matched successfully>

**Evaluation**

In [16]:
import os

old_folder_name = '/content/data/gtsrb/GTSRB/Training'
new_folder_name = '/content/data/gtsrb/GTSRB/Trainingg'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

old_folder_name = '/content/data/gtsrb/GTSRB/Testing'
new_folder_name = '/content/data/gtsrb/GTSRB/Training'

# Rename the folder
os.rename(old_folder_name, new_folder_name)

In [17]:
n_test_tasks = 1000

# test_set = CUB(split="test", training=False)

test_set = torchvision.datasets.GTSRB(
    root='/content/data',
    split='test',
    transform=torchvision.transforms.Compose(
        [
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Resize([image_size,image_size]),
        ]
    ),
    target_transform=None,
    download=True
    )

test_set.get_labels = lambda: [
    instance[1] for instance in test_set
]

test_sampler = TaskSampler(
    test_set, n_way=n_way, n_shot=n_shot, n_query=n_query, n_tasks=n_test_tasks
)
test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=n_workers,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

100%|██████████| 88978620/88978620 [00:02<00:00, 38576887.71it/s]


Extracting /content/data/gtsrb/GTSRB_Final_Test_Images.zip to /content/data/gtsrb


100%|██████████| 99620/99620 [00:00<00:00, 1473663.18it/s]

Extracting /content/data/gtsrb/GTSRB_Final_Test_GT.zip to /content/data/gtsrb


In [18]:
model.set_use_fc(False)

accuracy = evaluate(few_shot_classifier, test_loader, device=DEVICE)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 1000/1000 [01:04<00:00, 15.48it/s, accuracy=0.966]

Average accuracy : 96.61 %
